# record_digit_data
Let the user draw to define new digits and add data to the file digit_data.txt.

In [1]:
import tkinter as tk

In [2]:
from functools import partial

# Geometry constants.
NUM_ROWS = 8
NUM_COLS = 6
CELL_WID = 20
CELL_HGT = CELL_WID
MARGIN = 5
WINDOW_WID = NUM_COLS * CELL_WID * 2
WINDOW_HGT = NUM_ROWS * CELL_HGT + 20

class App:
    # Create and manage the tkinter interface.
    def __init__(self):
        self.network = None

        # Make the main interface.
        self.window = tk.Tk()
        self.window.title('record_digit_data')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry(f'{WINDOW_WID}x{WINDOW_HGT}')

        # Initially we have nothing to draw.
        self.polyline = None
        self.points = []
        self.digit = -1

        # Build the rest of the UI.
        self.build_ui()

        # Display the window.
        self.window.focus_force()
        self.window.mainloop()

    def redraw(self):
        # Remove old polyline.
        self.canvas.delete(self.polyline)
        self.polyline = None

        # Draw current points.
        if len(self.points) > 1:
            self.polyline = self.canvas.create_line(self.points, fill='black')

    def build_ui(self):
        # Make the drawing canvas.
        canvas_wid = NUM_COLS * CELL_WID + 1
        canvas_hgt = NUM_ROWS * CELL_HGT + 1
        self.canvas = tk.Canvas(self.window, bg='white',
            borderwidth=0, highlightthickness=0, relief=tk.SUNKEN, width=canvas_wid, height=canvas_hgt)
        self.canvas.pack(side=tk.LEFT, padx=MARGIN, pady=MARGIN)
        self.canvas.bind('<Button-1>', self.start_draw)
        self.canvas.bind('<ButtonRelease-1>', self.end_draw)

        # Make grid lines.
        for r in range(NUM_ROWS + 1):
            self.canvas.create_line(0, r * CELL_HGT, canvas_wid, r * CELL_HGT, fill='lime')
        for c in range(NUM_COLS + 1):
            self.canvas.create_line(c * CELL_WID, 0, c * CELL_WID, canvas_hgt, fill='lime')

        # Frame 1.
        self.buttons = []
        frame1 = tk.Frame(self.window)
        frame1.pack(side=tk.LEFT, padx=(MARGIN, 2))
        for i in range(5):
            self.buttons.append(tk.Button(frame1, text=f'{i}', width=2, relief='raised',
                command=partial(self.button_pressed, i)))
            self.buttons[i].pack(side=tk.TOP, pady=(0, 2))

        # Frame 2.
        frame2 = tk.Frame(self.window)
        frame2.pack(side=tk.LEFT)
        for i in range(5, 10):
            self.buttons.append(tk.Button(frame2, text=f'{i}', width=2, relief='raised',
                command=partial(self.button_pressed, i)))
            self.buttons[i].pack(side=tk.TOP, pady=(0, 2))

    def button_pressed(self, i):
        # Start drawing a new polyline.
        # Record the digit number.
        self.digit = i
        self.buttons[self.digit].configure(bg='pink', relief=tk.SUNKEN)

        # Remove any previous polyline.
        self.canvas.delete(self.polyline)

    def start_draw(self, event):
        if self.digit < 0:
            return
        self.canvas.bind('<B1-Motion>', self.save_point)

    def end_draw(self, event):
        if self.digit < 0:
            return
        self.canvas.unbind('<B1-Motion>')
        self.buttons[self.digit].configure(relief=tk.RAISED)

        # Save the polyline.
        self.save_polyline()

        # Prepare for the next one.
        self.digit = -1
        self.points = []
        self.redraw()

    def save_polyline(self):
        # Convert the points into the cells that were touched.
        touched = self.get_touched()

        # Save the touch data.
        with open('digit_data.txt', 'a') as f:
            f.write(f'{self.digit}: {self.touched_to_string(touched)}\n')
        print(f'Saved {self.digit}')

    # Convert the points into the cells that were touched.
    def get_touched(self):
        # Make a touched array holding 0s.
        touched = []
        for r in range(NUM_ROWS):
            touched.append([0 for i in range(NUM_COLS)])

        # Mark the touched cells.
        for point in self.points:
            r = int(point[1] / CELL_HGT)
            c = int(point[0] / CELL_WID)
            if r >= 0 and r < NUM_ROWS and c >= 0 and c < NUM_COLS:
                touched[r][c] = 1

        # Return the touched list.
        return touched
        #self.dump_touched(touched)

    # Return a string holding the touch values.
    def touched_to_string(self, touched):
        result = ''
        for r in range(NUM_ROWS):
            for c in range(NUM_COLS):
                result += str(touched[r][c])
        return result

    def dump_touched(self, touched):
        for r in range(NUM_ROWS):
            for c in range(NUM_COLS):
                if touched[r][c] == 1:
                    print(f'*', end='')
                else:
                    print(f' ', end='')
            print('')
        print('')

    def save_point(self, event):
        self.points.append((event.x, event.y))
        self.redraw()

    def kill_callback(self):
        self.window.destroy()

In [3]:
App()